# Text generation using a RNN

Given a sequence of words from this data, train a model to predict the next word in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

**Mount your Google Drive**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
import pickle 

from google.colab import drive 
drive.mount('/content/drive/')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [7]:
tf.keras.backend.clear_session()
# !pip install keras

### Import Keras and other libraries

In [8]:
from tensorflow import keras
from tensorflow import constant_initializer, convert_to_tensor 
from tensorflow.keras import layers, Sequential, Input,  Model, metrics
from tensorflow.keras.layers import Dense, Softmax, LSTM, Bidirectional
from tensorflow.keras.models import load_model
from tensorflow.keras import preprocessing 




## Download data
Reference: Data is collected from http://www.gutenberg.org

For the lab purpose, you can load the dataset provided by Great Learning

### Load the Oscar Wilde dataset

Store all the ".txt" file names in a list

In [9]:

#For displaying the presence of the files inside the library 
os.listdir('/content/drive/My Drive/sequential_nlp_data/')

# os.listdir('F:/Sequential NLP/data/data/')

['A House of Pomegranates.txt',
 'Oscar Wilde Miscellaneous.txt',
 'The Duchess of Padua.txt',
 'Selected poems of oscar wilde including The Ballad of Reading Gaol.txt',
 'Shorter Prose Pieces.txt',
 'For Love of the King.txt',
 'Poems with the Ballad of Reading Gaol.txt',
 'Salomé A tragedy in one act.txt',
 'The Ballad of Reading Gaol.txt',
 'Rose Leaf and Apple Leaf.txt',
 'Miscellaneous Aphorisms_ The Soul of Man.txt',
 'Lord Arthur Savile_s Crime.txt',
 'An Ideal Husband.txt',
 'A Woman of No Importance a play.txt',
 'Children in Prison and Other Cruelties of Prison Life.txt',
 'Impressions of America.txt',
 'Vera or, The Nihilists.txt',
 'Charmides and Other Poems.txt',
 'The Canterville Ghost.txt',
 'The Happy Prince and other tales.txt',
 'Essays and Lectures.txt',
 'Reviews.txt',
 'Miscellanies.txt',
 'The Importance of Being Earnest.txt',
 'De Profundis.txt',
 'Intentions.txt',
 'Lady Windermere_s Fan.txt',
 'Selected prose of oscar wilde with a Preface by Robert Ross.txt',


In [10]:


# Storing the text files data inside the list - text_files_data
text_files_data = os.listdir('/content/drive/My Drive/sequential_nlp_data/')

### Read the data

Read contents of every file from the list and append the text in a new list

In [11]:
# for accessing single file and print the contents inside it 
for content in open('/content/drive/My Drive/sequential_nlp_data/Reviews.txt', 'r'):
  print(content)

Streaming output truncated to the last 5000 lines.

   the darkness, ivories brown with age, worn brocades with gold threads

   gleaming in them, and tapestries with strange and pallid figures of

   dead gods, were all half revealed and half obscured in the twilight.

   As he moved through them, a figure which looked almost as pale as the

   Adonis of the tapestry and was erect and motionless like the statue of

   the wounded Love, came before his sight out of the darkness.  It was

   that of Gladys.



It is a manner full of exaggeration and overemphasis, but with some

remarkable rhetorical qualities and a good deal of colour.  Ouida is fond

of airing a smattering of culture, but she has a certain intrinsic

insight into things and, though she is rarely true, she is never dull.

Guilderoy, with all its faults, which are great, and its absurdities,

which are greater, is a book to be read.



Guilderoy.  By Ouida.  (Chatto and Windus.)









SOME LITERARY NOTES--VI





(Wom

In [12]:
# for accessing single file and print the contents inside it 
# for content in open('/content/drive/My Drive/NLP-July/data/Reviews.txt', 'r'):
#     print(len(content))
# This code could fetch words that are present in one line 

In [13]:
with open('/content/drive/My Drive/sequential_nlp_data/Reviews.txt', 'r') as f:
  content = f.readlines()
  content = [x.strip() for x in content]  # Helps in removing the '\n' in every line end 
    # the above code is done because, each line in the text is terminated with a special character called 
    # EOL(End of Line)
  
  print('Total lines that are present inside a Reviews.txt file =  ', len(content))
  print('Fetching first line of the given file : ',content[0])
  print('Count of letters in the first line: ', len(content[0]))
  #print(content)

Total lines that are present inside a Reviews.txt file =   18031
Fetching first line of the given file :  ﻿The Project Gutenberg eBook, Reviews, by Oscar Wilde, Edited by Robert Ross
Count of letters in the first line:  76


In [14]:


# words_from_every_file = []
# for files in text_files_data: 
#   with open('/content/drive/My Drive/NLP-July/data/'+str(files)) as f: 
#     #print(f.readlines())
#     content = f.readlines()
#     content = [x.strip() for x in content]
#     words_from_every_file.append(content)

words_from_every_file = []
for files in text_files_data: 
  content = open('/content/drive/My Drive/sequential_nlp_data/'+files,'r', errors = 'ignore') 
  content = content.read()
  # content = content.readlines() --> This completes every single line as the string with \n at the end 
  #content = [x.strip() for x in content]
  words_from_every_file.append(content)
  

words_from_every_file[:5]


['\ufeffThe Project Gutenberg eBook, A House of Pomegranates, by Oscar Wilde\n\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions \nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at \nwww.gutenberg.org.  If you are not located in the United States, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\n\n\n\nTitle: A House of Pomegranates\n\n\nAuthor: Oscar Wilde\n\n\n\nRelease Date: October 26, 2014  [eBook #873]\n[This file was first posted on April 8, 1997]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK A HOUSE OF POMEGRANATES***\n\n\nTranscribed from the 1915 Methuen and Co. edition by David Price, email\nccx074@pglaf.org\n\n                          [Picture: Book cover]\n\n                            

In [15]:
# def is_ascii(s):
#     return all(ord(c) < 128 for c in s)

# def is_ascii(s):
#     for file in words_from_every_file:
#         for character in file: 
#             print(ord(character) < 128)

# is_ascii(words_from_every_file[:1])



# list(words_from_every_file[:1])

In [16]:
print(len(words_from_every_file))

32


In [17]:
# Looking at first file words
print(type(words_from_every_file))
print(np.shape(words_from_every_file))

<class 'list'>
(32,)


In [18]:
len(words_from_every_file)

32

In [19]:
#words_from_every_file= [x.strip('\n') for x in words_from_every_file]
# words_from_every_file

In [20]:
tf.keras.backend.clear_session()

## Process the text
Initialize and fit the tokenizer

In [21]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\nâ€™S â€˜',
                                                                      lower=True,
                                                                      split=' ')
tokenizer.fit_on_texts(words_from_every_file[0:5])
# Indexing or vectorization of words is done 

word_index=tokenizer.word_index
# word_index

In [22]:

len(word_index)

9215

In [23]:
print(word_index)

{'the': 1, 'and': 2, 'of': 3, 'to': 4, 'a': 5, 'in': 6, 'i': 7, 'is': 8, 'that': 9, 'with': 10, 'he': 11, 'it': 12, 'not': 13, 'you': 14, 'for': 15, 'his': 16, 'or': 17, 'was': 18, 'him': 19, 'this': 20, 'but': 21, 'my': 22, 'as': 23, 'on': 24, 'from': 25, 'at': 26, 'me': 27, 'by': 28, 'be': 29, 'are': 30, 'they': 31, 'have': 32, 'will': 33, '’': 34, 'her': 35, 'gutenberg': 36, 'we': 37, 'all': 38, 'no': 39, 'who': 40, 'so': 41, 'guido': 42, 'project': 43, 'had': 44, 'their': 45, 'there': 46, 'thou': 47, 'when': 48, 'one': 49, 'she': 50, 'do': 51, 'your': 52, 'which': 53, 'what': 54, 'if': 55, 'any': 56, 'them': 57, 'love': 58, 'duchess': 59, 'were': 60, 'tm': 61, 'like': 62, 'its': 63, 'some': 64, 'said': 65, 'work': 66, 'thee': 67, 'man': 68, 'out': 69, 'upon': 70, 'lord': 71, 'little': 72, 'would': 73, '1': 74, 'up': 75, 'us': 76, 'may': 77, 'has': 78, 'into': 79, 'an': 80, 'thy': 81, 'other': 82, 'young': 83, 'our': 84, 'through': 85, 'should': 86, 'than': 87, 'soul': 88, 'can': 89

In [24]:
# Text to Sequence Conversion 

sequences_from_text=tokenizer.texts_to_sequences(words_from_every_file[0:5])
# sequences_from_text

In [25]:
len(sequences_from_text)

5

In [27]:
# Text to Matrix Convertion 

text_to_matrix = tokenizer.texts_to_matrix(words_from_every_file[0:5])
print("Dimension of the matrix formed", text_to_matrix.shape)
# text_to_matrix

Dimension of the matrix formed (5, 9216)


### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping words to numbers, and another for numbers to words.

In [28]:
# string_to_numerical1 = tokenizer.texts_to_sequences(words_from_every_file)
# numerical_to_string1 = tokenizer.sequences_to_texts(string_to_numerical)
# numerical_to_string1[30]

In [29]:
string_to_numerical = tokenizer.word_index
numerical_to_string = tokenizer.index_word
print("String mapped to Numerical for clearly  = ",string_to_numerical['clearly'])
print("Sequence number mapped to string for 10 = ",numerical_to_string[10])

String mapped to Numerical for clearly  =  1231
Sequence number mapped to string for 10 =  with


Get the word count for every word and also get the total number of words.

In [30]:
wordcount_for_every_word = tokenizer.word_counts       # type : collections.OrderedDict
count = 0
for x in wordcount_for_every_word:
  #a.append(wordcount_for_every_word[x])
  count = count + wordcount_for_every_word[x]

print("Total Numbner of words present in the vocublary :" , count)
print("Word count for each word : " , wordcount_for_every_word)

Total Numbner of words present in the vocublary : 107032
Word count for each word :  OrderedDict([('\ufeffthe', 4), ('project', 380), ('gutenberg', 412), ('ebook', 48), ('a', 2075), ('house', 86), ('of', 3352), ('pomegranates', 12), ('by', 498), ('oscar', 29), ('wilde', 40), ('this', 599), ('is', 1294), ('for', 907), ('the', 6543), ('use', 70), ('anyone', 20), ('anywhere', 15), ('in', 1626), ('united', 63), ('states', 80), ('and', 4326), ('most', 106), ('other', 173), ('parts', 9), ('world', 109), ('at', 542), ('no', 390), ('cost', 17), ('with', 1137), ('almost', 26), ('restrictions', 8), ('whatsoever', 10), ('you', 913), ('may', 192), ('copy', 59), ('it', 1045), ('give', 107), ('away', 104), ('or', 609), ('re', 9), ('under', 46), ('terms', 86), ('license', 65), ('included', 18), ('online', 16), ('www', 40), ('org', 52), ('if', 279), ('are', 490), ('not', 977), ('located', 28), ("you'll", 8), ('have', 471), ('to', 2156), ('check', 18), ('laws', 61), ('country', 29), ('where', 121), ('b

Convert text to sequence of numbers

In [31]:
text_to_sequence_of_numbers = tokenizer.texts_to_sequences(words_from_every_file[0:5])
# text_to_sequence_of_numbers

In [32]:
print(len(text_to_sequence_of_numbers))

5


In [33]:
tf.keras.backend.clear_session()

### Generate Features and Labels

In [34]:
input_data = []
output_data = []


for i in text_to_sequence_of_numbers:  # 5 array values 
  for j in range(0,len(i) - 10):
    input_seq  = i[j : j + 10]
    output_seq = i[j + 10]

    input_data.append(input_seq)
    output_data.append(output_seq)

  

print('Total number of input arrays: ', len(input_data))
print('Total number of Output arrays: ', len(output_data))
print("Input Data length: ",len(input_data[10]))


Total number of input arrays:  106982
Total number of Output arrays:  106982
Input Data length:  10


In [35]:
# input_output_list = list(zip(input_data, output_data))
# input_data_pickled, output_data_pickled =zip(*input_output_list)

In [36]:
# pickle_input = open('/content/drive/My Drive/NLP-July/'+'input_data_pickled.pickle', 'rb')
# input_data_picked = pickle.load(pickle_input)
# pickle_input.close()

# pickle_output = open('/content/drive/My Drive/NLP-July/'+'output_data_pickled.pickle', 'rb')
# output_data_pickled = pickle.load(pickle_out)
# pickle_output.close()

### The prediction task

Given a word, or a sequence of words, what is the most probable next word? This is the task we're training the model to perform. The input to the model will be a sequence of words, and we train the model to predict the output—the following word at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the words computed until this moment, what is the next word?

### Generate training and testing data

In [37]:
tf.keras.backend.clear_session()
# word_index = 13469
y = tf.keras.utils.to_categorical(output_data,num_classes = len(word_index)+1)

In [38]:
# pickle_input = open('F:/Sequential NLP/data/data/'+'input_to_pickle.pickle','wb')
# pickle.dump(input_data, pickle_input)
# pickle_input.close()

# pickle_output= open('F:/Sequential NLP/data/data/'+'output_to_pickle.pickle','wb')
# pickle.dump(y, pickle_output)
# pickle_output.close()
len(word_index)

9215

In [39]:
# y = tf.keras.utils.to_categorical(output_data,num_classes = len(word_index)+1)

In [40]:
# word_index = 13469
# y = tf.keras.utils.to_categorical(output_data,num_classes = (word_index)+1)
# # gc.collect()

In [41]:
# Need to perform only once



# pickle_input = open('/content/drive/My Drive/NLP-July/'+'input_to_pickle.pickle','wb')
# pickle.dump(input_data, pickle_input)
# pickle_input.close()

# pickle_output= open('/content/drive/My Drive/NLP-July/'+'output_to_pickle.pickle','wb')
# pickle.dump(y, pickle_output)
# pickle_output.close()

In [42]:
# input_data1 = open('input_to_pickle.pickle','rb')
# input_data_final = pickle.load(input_data1)

# output_data1 = open('output_to_pickle.pickle','rb')
# output_data_final = pickle.load(output_data1)

In [43]:
#X_train, X_test, y_train, y_test = train_test_split(input_data_final, y, test_size=0.30, random_state=42)

In [44]:
X_train,X_test, y_train, y_test = train_test_split(input_data,y, test_size=0.30, random_state=42)

In [45]:
np.array(X_train).shape

(74887, 10)

In [46]:
np.array(X_test).shape

(32095, 10)

In [47]:
y_train.shape

(74887, 9216)

In [48]:
y_test.shape

(32095, 9216)

In [49]:
import gc
input_data = None
y = None
gc.collect()

102

This is just to check the features and labels

In [50]:
#word_index = 13469
# y_train = tf.keras.utils.to_categorical(y_train,num_classes = len(word_index)+1)
# y_test = tf.keras.utils.to_categorical(y_test,num_classes = len(word_index)+1)

In [51]:
# pickle_input = open('/content/drive/My Drive/NLP-July/'+'input_to_pickle.pickle','wb')
# pickle.dump(input_data, pickle_input)
# pickle_input.close()

# pickle_output= open('/content/drive/My Drive/NLP-July/'+'output_to_pickle.pickle','wb')
# pickle.dump(y, pickle_output)
# pickle_output.close()

## Build The Model

Use `keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `keras.layers.LSTM`: A type of RNN with size `units=rnn_units` (You can also use a GRU layer here.)
* `keras.layers.Dense`: The output layer, with `num_words` outputs.

In [52]:
input_length = len(X_train[0])
input_length

10

In [74]:
input_length = 15
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = len(word_index)+1, 
                                   output_dim = 300,
                                   input_length =input_length))
# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, activation = 'tanh',
#                                recurrent_activation='sigmoid')))

model.add(tf.keras.layers.LSTM(70, activation = 'tanh', recurrent_activation = 'sigmoid')) # return_sequences= True))
#model.add(tf.keras.layers.LSTM(200, activation = 'tanh', recurrent_activation = 'sigmoid', return_sequences= True))
# model.add(tf.keras.layers.Dense(32, activation = 'relu'))


#model.add(tf.keras.layers.LSTM(100, activation = 'tanh', recurrent_activation ='sigmoid'))


model.add(tf.keras.layers.Dense(len(word_index)+1,activation = 'softmax'))


model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy']   )
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 300)           2764800   
_________________________________________________________________
lstm (LSTM)                  (None, 70)                103880    
_________________________________________________________________
dense (Dense)                (None, 9216)              654336    
Total params: 3,523,016
Trainable params: 3,523,016
Non-trainable params: 0
_________________________________________________________________
None


In [75]:
tf.keras.backend.clear_session()

For each word the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next word.

## Train the model

In [76]:
# Saving the weights for the model during training

trained_weights  = ''

In [77]:
model.fit(np.array(X_train), y_train, batch_size= 30, epochs = 10, verbose = 1)

Epoch 1/10
2497/2497 [==============================] - 122s 49ms/step - loss: 6.7328 - accuracy: 0.0778
Epoch 2/10
2497/2497 [==============================] - 122s 49ms/step - loss: 5.9033 - accuracy: 0.1229
Epoch 3/10
2497/2497 [==============================] - 121s 48ms/step - loss: 5.3390 - accuracy: 0.1557
Epoch 4/10
2497/2497 [==============================] - 121s 48ms/step - loss: 4.8964 - accuracy: 0.1869
Epoch 5/10
2497/2497 [==============================] - 122s 49ms/step - loss: 4.4996 - accuracy: 0.2222
Epoch 6/10
2497/2497 [==============================] - 121s 49ms/step - loss: 4.1325 - accuracy: 0.2607
Epoch 7/10
2497/2497 [==============================] - 123s 49ms/step - loss: 3.7900 - accuracy: 0.3014
Epoch 8/10
2497/2497 [==============================] - 122s 49ms/step - loss: 3.4720 - accuracy: 0.3441
Epoch 9/10
2497/2497 [==============================] - 121s 49ms/step - loss: 3.1788 - accuracy: 0.3891
Epoch 10/10
2497/2497 [==============================] 

In [79]:
model.evaluate(np.array(X_test), y_test, batch_size = 32)

1003/1003 [==============================] - 8s 8ms/step - loss: 6.3997 - accuracy: 0.1994


[6.399679183959961, 0.1994391679763794]

In [80]:
X_test = np.array(X_test)
for i, j in enumerate(X_test[2010:2015],start = 2010):
    text = []
    for k in j:
        text.append(numerical_to_string[k])
    
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + numerical_to_string[np.argmax(y_test[i])] + '\n')
    model_pred = model.predict(np.array(X_test))[i]
    print('Predicated word: ' + numerical_to_string[np.argmax(model_pred)])

Features: to each other and began to quarrel over a string

Label: of

Predicated word: or
Features: utility and the demands of good taste the sarah bernhardt

Label: tea

Predicated word: software
Features: above the din and fray we heard the loud musicians

Label: play

Predicated word: and
Features: chamber and leave this letter and this dagger there that

Label: when

Predicated word: is
Features: got to swing ’ dear christ the very prison walls

Label: suddenly

Predicated word: is


### Save Model

In [ ]:
# model.save('/content/drive/My Drive/sequential_nlp_data/sequential_nlp_data_weights'+'my_model_stack2LSTM_Dense.h5')

## If you have already trained the model and saved it, you can load a pretrained model

In [ ]:
# model1 = tf.keras.models.load_model('/content/drive/My Drive/sequential_nlp_data_weights/'+'sequential_nlp_data_weightsmy_model_biLSTM_Dense.h5')

### Note: After loading the model run  model.fit()  to continue training form there, if required.

In [ ]:
# model1.fit(np.array(X_train), y_train, batch_size = 50, epochs = 20, verbose = 1)

Epoch 1/20
127298/127298 [==============================] - 65s 514us/sample - loss: 0.2638 - acc: 0.9386
Epoch 2/20
127298/127298 [==============================] - 55s 435us/sample - loss: 0.2595 - acc: 0.9388
Epoch 3/20
127298/127298 [==============================] - 56s 440us/sample - loss: 0.2497 - acc: 0.9413 - loss
Epoch 4/20
127298/127298 [==============================] - 56s 441us/sample - loss: 0.2458 - acc: 0.9420
Epoch 5/20
127298/127298 [==============================] - 88s 694us/sample - loss: 0.2399 - acc: 0.9433
Epoch 6/20
127298/127298 [==============================] - 111s 870us/sample - loss: 0.2406 - acc: 0.9428
Epoch 7/20
127298/127298 [==============================] - 106s 830us/sample - loss: 0.2281 - acc: 0.9457
Epoch 8/20
127298/127298 [==============================] - 104s 818us/sample - loss: 0.2299 - acc: 0.9443
Epoch 9/20
127298/127298 [==============================] - 119s 935us/sample - loss: 0.2210 - acc: 0.9470- loss: 0.2209 - acc
Epoch 10/20
127

In [ ]:
model.evaluate(X_test, y_test, batch_size = 124)

259/259 [==============================] - 12s 45ms/step - loss: 9.1287 - accuracy: 1.2463e-04


[9.128668785095215, 0.00012463000894058496]

## Evaluation

In [ ]:
X_test = np.array(X_test)
for i, j in enumerate(X_test[100:105],start = 1):
    text = []
    for k in j:
        text.append(numerical_to_string[k])
    
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + numerical_to_string[np.argmax(y_test[i])] + '\n')
    model_pred = model1.predict(np.array(X_test))[i]
    print('Predicated word: ' + numerical_to_string[np.argmax(model_pred)])

Features: comply with both paragraphs 1 e 1 through 1 e

Label: literary

Predicated word: boy
Features: solemnly in to a little gilt and ivory chair that

Label: canvas

Predicated word: gold
Features: more duchess better for me i had not seen your

Label: our

Predicated word: dead
Features: ducal palace a window l c looks out on a

Label: is

Predicated word: is
Features: for three hundred wax lights hung down from the black

Label: will

Predicated word: have


In [ ]:
loss_train, acc_train = model1.evaluate(np.array(X_train),y_train, verbose = 2)
print("Accuracy on training data : {:5.2f}%".format(acc_train%100))
print("Loss on training data : {:5.2f}%".format(loss_train))

127298/127298 - 65s - loss: 0.1468 - acc: 0.9660
Accuracy on training data :  0.97%
Loss on training data :  0.15%


In [ ]:
loss_test, acc_test = model1.evaluate(np.array(X_test),y_test, verbose = 1)
print("Accuracy on testing data : {:5.2f}%".format(acc_test%100))
print("Loss on testing data : {:5.2f}%".format(loss_test))

54557/54557 [==============================] - 30s 549us/sample - loss: 12.7523 - acc: 0.1622
Accuracy on testing data :  0.16%
Loss on testing data : 12.75%


## Generate text

##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



---



---

